# Importing, libraries, and formating:

In [11]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
# import scipy.stats as st
# import plotly.express as px

In [12]:
# importing and basic formating
flights = pd.read_csv('C:/Users/danfv/OneDrive/Knowledge/Courses/Data Analysis/LHL - Data Analysis Bootcamp/lighthouse-data-notes/W6 Midterm Project/Midterm_Project/Data/Flights - Sample 50000 - Added Dan Features.csv').sort_values('fl_date').reset_index(drop=True)
flights = flights.drop('Unnamed: 0', axis=1)

# deleting features the model won't use
drop_engineered_dan = ['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'op_carrier_fl_num',
    'crs_dep_time', 'crs_arr_time', 'month']
drop_engineered_peter = ['origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name']
drop_streamlining = ['tail_num', 'dup', 'flights']
original_remain = ['crs_elapsed_time', 'distance']

initial_drop = drop_engineered_dan + drop_engineered_peter + drop_streamlining
flights = flights.drop(initial_drop, axis=1)

# renaming target
flights = flights.rename(columns={'total_delay' : 'arr_delay'})

# Separating features and target for regression:
X = flights.drop('arr_delay', axis=1)
y = flights.arr_delay


In [41]:
X

flights

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,total_delay,Season,Weekday,month,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,carrier_enc,airtime_ind,orig_airp_fl_ind,dest_airp_fl_ind,orig_airp_pss_ind,dest_airp_pss_ind,orig_airp_enc,dest_airp_enc
0,2018-01-01,DL,DL,DL,260,DL,N964DN,260,14869,SLC,"Salt Lake City, UT",13487,MSP,"Minneapolis, MN",1715,2057,N,162,1,991,0.0,1,0,1,0,0.061765,0.053236,-0.019545,3,0.225312,0.01512,0.01992,0.013807,0.023263,322,239
1,2018-01-01,UA,UA_CODESHARE,UA,5510,OO,N905SW,5510,13367,MLI,"Moline, IL",11292,DEN,"Denver, CO",1655,1820,N,145,1,752,0.0,1,0,1,0,0.056478,0.059659,0.055916,8,0.225312,0.00074,0.03440,0.000927,0.030772,227,93
2,2018-01-01,WN,WN,WN,491,WN,N231WN,491,13931,ORF,"Norfolk, VA",10821,BWI,"Baltimore, MD",1535,1630,N,55,1,159,32.0,1,0,1,0,0.059419,0.056478,-0.037253,10,0.213424,0.00346,0.01352,0.003075,0.010324,250,58
3,2018-01-01,AA,AA,AA,2185,AA,N896NN,2185,13930,ORD,"Chicago, IL",13487,MSP,"Minneapolis, MN",2049,2214,N,85,1,334,0.0,1,0,1,0,0.053236,0.032434,-0.001450,0,0.350266,0.04976,0.01992,0.042113,0.023263,249,239
4,2018-01-01,AA,AA_CODESHARE,AA,3781,MQ,N243NN,3781,13871,OMA,"Omaha, NE",13930,ORD,"Chicago, IL",925,1101,N,96,1,416,42.0,1,0,1,0,0.062352,0.058166,-0.001450,0,0.350266,0.00364,0.04976,0.004162,0.042113,246,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2019-12-31,AA,AA_CODESHARE,AA,4595,YX,N443YX,4595,13303,MIA,"Miami, FL",15412,TYS,"Knoxville, TN",1541,1756,N,135,1,724,0.0,1,1,12,1,0.059419,0.061765,-0.001450,0,0.225312,0.01078,0.00266,0.023044,0.002415,222,355
49996,2019-12-31,UA,UA_CODESHARE,UA,4518,G7,N543GJ,4518,11003,CID,"Cedar Rapids/Iowa City, IA",13930,ORD,"Chicago, IL",1515,1635,N,80,1,196,74.0,1,1,12,1,0.059419,0.056478,0.055916,8,0.350266,0.00148,0.04976,0.001873,0.042113,67,252
49997,2019-12-31,DL,DL,DL,976,DL,N972AT,976,10397,ATL,"Atlanta, GA",12266,IAH,"Houston, TX",1351,1515,N,144,1,689,0.0,1,1,12,1,0.054044,0.059419,-0.019545,3,0.225312,0.04904,0.02232,0.035133,0.024860,21,166
49998,2019-12-31,WN,WN,WN,5141,WN,N8689C,5141,12889,LAS,"Las Vegas, NV",14683,SAT,"San Antonio, TX",2030,55,N,145,1,1069,0.0,1,1,12,1,0.053236,0.008157,-0.037253,10,0.225312,0.02098,0.00524,0.015935,0.006244,187,304


# Defining Function to run linear regressions:

In [13]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor

In [14]:
regressor_list = ['LinearRegression()', 'RandomForestRegressor()']
func_list = ['None', 'np.log']
inverse_func_list = ['None', 'np.exp']

def trans_tar_reg_function(X, y, regressor, func, inverse_func):
    '''
    Generates R2, RMSE and MAE of models using the specified regressor, function and inverse_func for y
    Parameters:
    - X = features df
    - y = target df
    - regressor = method/algo to do the regression
    - func = transformation of y
    - inverse_fun = to get transformed y back to original units
    '''

    #instantiating the target regressor
    ttr = TransformedTargetRegressor(regressor=regressor, func=func, inverse_func=inverse_func)
    # fitting 
    ttr.fit(X, y)
    # Model Evaluation
    y_pred = ttr.predict(X)
    n = y.shape[0]
    p = X.shape[1]
    R2_adj = 1-(1-r2_score(y, y_pred))*(n-1)/(n-p-1)
    RMSE = mean_squared_error(y, y_pred)
    MAE = mean_absolute_error(y, y_pred)

    return [R2_adj, RMSE, MAE]

# Feature Selection

## Original features:

In [15]:
# original X:
X

# Removing features with p-value > 0.05 when fitting with X (prob. weight = 0 is statistically significant):
X_pval_filter = X.drop(['Season', 'Weekday', 'dest_airp_fl_ind',
       'orig_airp_pss_ind', 'orig_airp_pss_ind', 'dest_airp_pss_ind'], axis=1)

## Standardizing features that were in original units:

In [16]:
# adding 1 to zero values in y (flights with no delays):
y_plus1 = y + 1

# standardized time and distance:
from sklearn.preprocessing import StandardScaler
#new df with standardized values
X_stscaled = X.copy()
# features to standardize
stand_feat = ['crs_elapsed_time', 'distance']
# adding the standardized columns
standardized_feat = StandardScaler().fit_transform(X_stscaled[stand_feat])
X_stscaled['crs_elap_t_std'], X_stscaled['dist_std'] = [standardized_feat[:,0], standardized_feat[:,1]]
# dropping the original values
X_stscaled = X_stscaled.drop(['crs_elapsed_time', 'distance'], axis=1)

## Small Variance:

In [19]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(0.1)
high_var_arr = vt.fit_transform(X) # features with small variance get deleted

In [21]:
high_var_arr.shape # array of the ones that remained

(50000, 5)

In [22]:
# Creating the df back
X_highv_col = X.columns[vt.get_support()] # get_support() is method of VarianceThreshold and stores boolean of each variable in the numpy array.
X_high_var = pd.DataFrame(high_var_arr, columns=X_highv_col) # transforming an array back to a data-frame preserves column labels

In [23]:
X_high_var

,crs_elapsed_time,distance,Season,Weekday,peak_season
0,162.0,991.0,1.0,0.0,0.0
1,145.0,752.0,1.0,0.0,0.0
2,55.0,159.0,1.0,0.0,0.0
3,85.0,334.0,1.0,0.0,0.0
4,96.0,416.0,1.0,0.0,0.0
...,...,...,...,...,...
49995,135.0,724.0,1.0,1.0,1.0
49996,80.0,196.0,1.0,1.0,1.0
49997,144.0,689.0,1.0,1.0,1.0
49998,145.0,1069.0,1.0,1.0,1.0


## Correlations:

In [37]:
flights.corr()

,crs_elapsed_time,distance,arr_delay,Season,Weekday,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,airtime_ind,orig_airp_fl_ind,dest_airp_fl_ind,orig_airp_pss_ind,dest_airp_pss_ind
crs_elapsed_time,1.000000,0.982811,0.011791,-0.019695,0.012763,0.004123,-0.015907,-0.102304,0.059660,-0.791081,0.076903,0.115792,0.115708,0.149139
distance,0.982811,1.000000,0.007873,-0.005110,0.013522,0.012390,-0.042415,-0.128124,0.033846,-0.792332,0.079169,0.074925,0.113168,0.108683
arr_delay,0.011791,0.007873,1.000000,-0.008931,0.002362,0.032897,0.007533,-0.044802,0.063860,-0.000548,-0.001038,0.010215,0.003695,0.015781
Season,-0.019695,-0.005110,-0.008931,1.000000,0.009396,-0.015854,0.000837,-0.002892,0.013351,0.013178,0.014371,-0.000686,0.014809,-0.000207
Weekday,0.012763,0.013522,0.002362,0.009396,1.000000,0.015406,0.008558,0.001976,0.009204,-0.007992,-0.003575,0.001827,-0.004315,0.002632
peak_season,0.004123,0.012390,0.032897,-0.015854,0.015406,1.000000,-0.016782,-0.016755,-0.003387,-0.010851,-0.003045,-0.001271,-0.000890,-0.001262
dep_taxi_ind,-0.015907,-0.042415,0.007533,0.000837,0.008558,-0.016782,1.000000,0.418970,0.006905,0.043551,0.028341,-0.032084,0.032464,-0.039296
arr_taxi_ind,-0.102304,-0.128124,-0.044802,-0.002892,0.001976,-0.016755,0.418970,1.000000,0.013738,0.097897,-0.129395,0.119425,-0.136606,0.108132
carriers_lateness_ind,0.059660,0.033846,0.063860,0.013351,0.009204,-0.003387,0.006905,0.013738,1.000000,-0.045075,0.077981,0.070116,0.140047,0.131613
airtime_ind,-0.791081,-0.792332,-0.000548,0.013178,-0.007992,-0.010851,0.043551,0.097897,-0.045075,1.000000,-0.059288,-0.047461,-0.087464,-0.079663


In [28]:
X_stscaled.corr()[X_stscaled.corr().abs() > 0.8]

,Season,Weekday,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,airtime_ind,orig_airp_fl_ind,dest_airp_fl_ind,orig_airp_pss_ind,dest_airp_pss_ind,crs_elap_t_std,dist_std
Season,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Weekday,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
peak_season,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dep_taxi_ind,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
arr_taxi_ind,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
carriers_lateness_ind,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
airtime_ind,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
orig_airp_fl_ind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,NaN,0.94868,NaN,NaN,NaN
dest_airp_fl_ind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.948303,NaN,NaN
orig_airp_pss_ind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.94868,NaN,1.00000,NaN,NaN,NaN


In [ ]:
# dropping features with high correlations in correlation matrix X vs y:
X_non_corr = X.drop(['distance', 'orig_airp_pss_ind', 'dest_airp_pss_ind'], axis=1)

# dropping features with high correlations in correlation matrix X_stscaled vs y:
X_stscaled_non_corr = X_stscaled.drop(['dist_std', 'orig_airp_pss_ind', 'dest_airp_pss_ind'], axis=1)

## 

## Feature Selection: Forward Regression

In [33]:
from sklearn.feature_selection import f_regression, SelectKBest
skb = SelectKBest(f_regression, k=10) # instantiating the n features that best explain y
X_fwd_reg_array = skb.fit_transform(X, y) #applying algo to the data
X_fwd_reg_array.shape

(50000, 10)

In [34]:
# creating a Datarame with the values of the matrix X and the column names above
X_fwd_reg = pd.DataFrame(X, columns=X.columns[skb.get_support()]) #get support: list that holds True if the feature was selected

In [35]:
X_fwd_reg

,crs_elapsed_time,distance,Season,peak_season,dep_taxi_ind,arr_taxi_ind,carriers_lateness_ind,dest_airp_fl_ind,orig_airp_pss_ind,dest_airp_pss_ind
0,162,991,1,0,0.061765,0.053236,-0.019545,0.01992,0.013807,0.023263
1,145,752,1,0,0.056478,0.059659,0.055916,0.03440,0.000927,0.030772
2,55,159,1,0,0.059419,0.056478,-0.037253,0.01352,0.003075,0.010324
3,85,334,1,0,0.053236,0.032434,-0.001450,0.01992,0.042113,0.023263
4,96,416,1,0,0.062352,0.058166,-0.001450,0.04976,0.004162,0.042113
...,...,...,...,...,...,...,...,...,...,...
49995,135,724,1,1,0.059419,0.061765,-0.001450,0.00266,0.023044,0.002415
49996,80,196,1,1,0.059419,0.056478,0.055916,0.04976,0.001873,0.042113
49997,144,689,1,1,0.054044,0.059419,-0.019545,0.02232,0.035133,0.024860
49998,145,1069,1,1,0.053236,0.008157,-0.037253,0.00524,0.015935,0.006244


# Adding iterations to df:

In [ ]:
# Creating empty df to store metrics and compare:

# lr_metrics_df = pd.DataFrame(columns=['R2_adj', 'RMSE', 'MAE'], dtype='float64')


In [ ]:
trans_tar_reg_function(X_stscaled_correl_filter, y_plus1, LinearRegression(), np.log, np.exp)
# trans_tar_reg_function(X_correl_filter, y, LinearRegression(), None, None)

[-0.05981474200934778, 2505.5137068687986, 13.791662512853026]

In [ ]:
lr_metrics_df.loc['X_stscaled_correl_filter / y_plus1'] = trans_tar_reg_function(X_stscaled_correl_filter, y_plus1, LinearRegression(), np.log, np.exp)

In [30]:
lr_metrics_df

NameError: name 'lr_metrics_df' is not defined

# Appends

## Statsmodel Regression for p-values:

In [ ]:
# adding a constant for X
X_all = sm.add_constant(X)

#modeling vanilla ls
model = sm.OLS(y, X_all).fit()
# pickle.dump(model, open('Data/Pickles/vanilla_ls_allfeat_noscalling.sav', 'wb'))
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              arr_delay   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     34.45
Date:                Wed, 22 Sep 2021   Prob (F-statistic):           4.28e-87
Time:                        17:25:55   Log-Likelihood:            -2.6493e+05
No. Observations:               50000   AIC:                         5.299e+05
Df Residuals:                   49986   BIC:                         5.300e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     9.55

In [ ]:
# Model Evaluation
y_pred = model.predict(X_all)
n = y.shape[0]
p = X_all.shape[1]
print('R2 (adj) =', 1-(1-r2_score(y, y_pred))*(n-1)/(n-p-1))
print('RMSE =', mean_squared_error(y, y_pred))
print('MAE =', mean_absolute_error(y, y_pred))


R2 (adj) = 0.008603122510426053
RMSE = 2343.579124388721
MAE = 20.949733276135145


## Random Forest Trial:

In [ ]:
# STANDARDIZING TARGET
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

X, y = make_regression(n_features=4, n_informative=2,
                       random_state=0, shuffle=False)
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X, y)

print(regr.predict([[0, 0, 0, 0]]))

[-8.32987858]


In [ ]:

y_pred = regr.predict(X)
n = y.shape[0]
p = X.shape[1]
print('R2 (adj) =', 1-(1-r2_score(y, y_pred))*(n-1)/(n-p-1))
print('RMSE =', mean_squared_error(y, y_pred))
print('MAE =', mean_absolute_error(y, y_pred))


R2 (adj) = 0.8369970731647233
RMSE = 232.4546273335677
MAE = 12.672871643060946
